# LangChain 1.0 Runtime Examples

This notebook demonstrates using Runtime in **Tools** and **Middleware (Hooks)**

## Key Concepts:
- **Context**: Static information like user ID, DB connections, API keys
- **Store**: Long-term memory for persisting data across conversations
- **Stream Writer**: Custom streaming for progress updates and events

## Setup
First, install required packages:
```bash
pip install langchain langgraph langchain-openai pydantic
```

## 1. Import Dependencies

In [ ]:
from typing import Annotated
from pydantic import BaseModel, Field
from langchain_core.tools import tool
from langgraph.prebuilt import create_agent
from langgraph.runtime import get_runtime, Runtime
from langgraph.store.base import BaseStore

## 2. Define Context Schema

The context schema defines what information is available during runtime

In [ ]:
class UserContext(BaseModel):
    """Context schema defining what information is available during runtime"""
    user_id: str = Field(description="User identifier")
    database_url: str = Field(description="Database connection string")
    organization_id: str = Field(description="Organization identifier")
    api_key: str = Field(description="API key for external services")

# Display the schema
print("Context Schema:")
print(UserContext.model_json_schema())

## 3. Tools with Runtime Access

### Example 1: Reading Context in Tools

In [ ]:
@tool
def get_user_preferences(preference_type: str) -> str:
    """
    Fetch user preferences from database using runtime context.
    
    Args:
        preference_type: Type of preference to fetch (e.g., 'theme', 'language')
    """
    # Access the runtime object
    runtime = get_runtime()
    
    # Access context information
    context = runtime.context
    user_id = context.get("user_id")
    db_url = context.get("database_url")
    
    # Simulate database query
    print(f"Querying database at {db_url} for user {user_id}")
    
    # Return mock data
    return f"User {user_id} preference for {preference_type}: dark_mode"

# Test the tool definition
print(f"\nTool: {get_user_preferences.name}")
print(f"Description: {get_user_preferences.description}")

### Example 2: Using Store (Long-term Memory)

In [ ]:
@tool
def save_conversation_summary(summary: str) -> str:
    """
    Save a conversation summary to long-term memory.
    
    Args:
        summary: Text summary to save
    """
    runtime = get_runtime()
    
    # Access the store for long-term memory
    store = runtime.store
    user_id = runtime.context.get("user_id")
    
    # Write to store with namespace
    namespace = ("conversations", user_id)
    key = f"summary_{runtime.context.get('session_id', 'default')}"
    
    # Put data in store
    store.put(namespace, key, {"summary": summary, "timestamp": "2025-10-21"})
    
    return f"Saved summary for user {user_id}"


@tool
def retrieve_past_summaries(limit: int = 5) -> str:
    """
    Retrieve past conversation summaries from long-term memory.
    
    Args:
        limit: Maximum number of summaries to retrieve
    """
    runtime = get_runtime()
    store = runtime.store
    user_id = runtime.context.get("user_id")
    
    # Search in store
    namespace = ("conversations", user_id)
    items = store.search(namespace, limit=limit)
    
    summaries = [item.value.get("summary") for item in items]
    return f"Found {len(summaries)} past summaries: {summaries}"

print("Tools for long-term memory created successfully!")

### Example 3: Custom Streaming from Tools

In [ ]:
@tool
def process_large_file(file_path: str) -> str:
    """
    Process a large file with progress updates via custom stream.
    
    Args:
        file_path: Path to the file to process
    """
    runtime = get_runtime()
    
    # Simulate processing with progress updates
    steps = ["Reading file", "Parsing data", "Analyzing content", "Generating report"]
    
    for i, step in enumerate(steps):
        # Write progress to custom stream
        runtime.stream_writer.write({
            "type": "progress",
            "step": step,
            "progress": (i + 1) / len(steps) * 100
        })
    
    return f"File {file_path} processed successfully"

print("Custom streaming tool created!")

### Example 4: Tool with API Key from Context

In [ ]:
@tool
def call_external_api(endpoint: str, data: dict) -> str:
    """
    Call external API using API key from context.
    
    Args:
        endpoint: API endpoint to call
        data: Data to send to API
    """
    runtime = get_runtime()
    
    # Get API key from context
    api_key = runtime.context.get("api_key")
    org_id = runtime.context.get("organization_id")
    
    if not api_key:
        return "Error: API key not found in context"
    
    # Simulate API call
    print(f"Calling {endpoint} with org {org_id}")
    print(f"Using API key: {api_key[:8]}...")
    
    return f"API call to {endpoint} successful"

print("API tool created!")

## 4. Middleware: Pre-Model Hooks

Pre-model hooks act as middleware that intercepts requests before sending to the LLM

### Example 5: Request Logging Hook

In [ ]:
def pre_model_logging_hook(runtime: Runtime, messages: list) -> list:
    """
    Pre-model hook that logs requests before sending to LLM.
    This acts like middleware, intercepting before model invocation.
    
    Args:
        runtime: Runtime object injected automatically
        messages: Messages to be sent to the model
    """
    user_id = runtime.context.get("user_id")
    
    # Log the request
    print(f"\n{'='*50}")
    print(f"PRE-MODEL HOOK: User {user_id} sending {len(messages)} messages")
    print(f"Last message: {messages[-1].content[:100]}...")
    
    # Write to custom stream
    runtime.stream_writer.write({
        "type": "model_call_start",
        "user_id": user_id,
        "message_count": len(messages)
    })
    
    # Can modify messages here if needed
    return messages

print("Pre-model logging hook defined!")

### Example 6: Rate Limiting Hook

In [ ]:
def rate_limiting_hook(runtime: Runtime, messages: list) -> list:
    """
    Pre-model hook implementing rate limiting per user.
    """
    user_id = runtime.context.get("user_id")
    store = runtime.store
    
    # Check rate limit from store
    namespace = ("rate_limits", user_id)
    rate_limit_data = store.get(namespace, "requests")
    
    if rate_limit_data:
        request_count = rate_limit_data.value.get("count", 0)
        if request_count > 100:  # Example limit
            raise Exception(f"Rate limit exceeded for user {user_id}")
        
        # Increment counter
        store.put(namespace, "requests", {"count": request_count + 1})
    else:
        # Initialize counter
        store.put(namespace, "requests", {"count": 1})
    
    print(f"Rate limit check passed for user {user_id}")
    return messages

print("Rate limiting hook defined!")

## 5. Middleware: Post-Model Hooks

Post-model hooks process responses after LLM generation

### Example 7: Response Processing Hook

In [ ]:
def post_model_processing_hook(runtime: Runtime, response: dict) -> dict:
    """
    Post-model hook that processes responses after LLM generation.
    
    Args:
        runtime: Runtime object injected automatically
        response: Response from the model
    """
    user_id = runtime.context.get("user_id")
    
    # Log the response
    print(f"\nPOST-MODEL HOOK: User {user_id} received response")
    
    # Save to long-term memory
    store = runtime.store
    namespace = ("model_responses", user_id)
    store.put(namespace, "last_response", {
        "content": str(response)[:200],
        "timestamp": "2025-10-21"
    })
    
    # Write to custom stream
    runtime.stream_writer.write({
        "type": "model_call_end",
        "user_id": user_id,
        "response_length": len(str(response))
    })
    
    return response

print("Post-model processing hook defined!")

## 6. Creating an Agent with Runtime Configuration

In [ ]:
def create_example_agent():
    """
    Example of creating an agent with runtime context and hooks.
    """
    from langchain_openai import ChatOpenAI
    
    # Define tools
    tools = [
        get_user_preferences,
        save_conversation_summary,
        retrieve_past_summaries,
        process_large_file,
        call_external_api
    ]
    
    # Create model
    model = ChatOpenAI(model="gpt-4")
    
    # Create agent with context schema and hooks
    agent = create_agent(
        model=model,
        tools=tools,
        context_schema=UserContext,
        # Add middleware-like hooks
        pre_model_hooks=[pre_model_logging_hook, rate_limiting_hook],
        post_model_hooks=[post_model_processing_hook]
    )
    
    return agent

print("Agent creation function defined!")
print("\nNote: Requires OPENAI_API_KEY environment variable to be set")

## 7. Invoking the Agent with Runtime Context

In [ ]:
def invoke_agent_example():
    """
    Example of invoking an agent with runtime context.
    """
    agent = create_example_agent()
    
    # Prepare context for this invocation
    context = {
        "user_id": "user_12345",
        "database_url": "postgresql://localhost:5432/mydb",
        "organization_id": "org_789",
        "api_key": "sk-abc123xyz",
        "session_id": "session_001"
    }
    
    # Invoke agent with context
    result = agent.invoke(
        {
            "messages": [
                {"role": "user", "content": "Get my theme preference and save this conversation"}
            ]
        },
        context=context  # Pass runtime context
    )
    
    return result

print("Agent invocation function defined!")

## 8. Streaming with Custom Updates

In [ ]:
import asyncio

async def streaming_example():
    """
    Example of streaming with custom updates from tools.
    """
    agent = create_example_agent()
    
    context = {
        "user_id": "user_12345",
        "database_url": "postgresql://localhost:5432/mydb",
        "organization_id": "org_789",
        "api_key": "sk-abc123xyz"
    }
    
    # Stream with custom mode to receive tool progress
    async for chunk in agent.astream(
        {"messages": [{"role": "user", "content": "Process large_file.csv"}]},
        context=context,
        stream_mode=["custom", "messages"]  # Enable custom stream mode
    ):
        if chunk.get("type") == "progress":
            print(f"Progress: {chunk['step']} - {chunk['progress']:.1f}%")
        elif chunk.get("type") == "model_call_start":
            print(f"Model call started for user {chunk['user_id']}")

print("Streaming example function defined!")

## 9. Run Examples

Uncomment and run the cells below to test the agent:

In [ ]:
# Set your OpenAI API key first
# import os
# os.environ["OPENAI_API_KEY"] = "your-api-key-here"

# Run the agent
# result = invoke_agent_example()
# print("\nAgent Result:")
# print(result)

## 10. Run Streaming Example

In [ ]:
# Run streaming example (in async context)
# await streaming_example()

# Or in a regular Python script:
# asyncio.run(streaming_example())

## Summary

This notebook demonstrated:

1. **Tools with Runtime Access**:
   - Reading context (user ID, database URLs, API keys)
   - Using Store for long-term memory
   - Custom streaming for progress updates

2. **Middleware (Hooks)**:
   - Pre-model hooks: Logging, rate limiting
   - Post-model hooks: Response processing, saving to memory

3. **Agent Creation & Invocation**:
   - Creating agents with context schema
   - Passing runtime context on invocation
   - Streaming with custom updates

### Key Takeaway Pattern:
```python
from langgraph.runtime import get_runtime

@tool
def my_tool():
    runtime = get_runtime()
    
    # Access context
    user_id = runtime.context.get("user_id")
    
    # Use store
    runtime.store.put(namespace, key, value)
    
    # Stream updates
    runtime.stream_writer.write({"type": "progress"})
```